# GPT API를 활용한 좋은 리뷰요약

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# 환경변수 로딩
load_dotenv('.env', override=True)

# 메모리에 로딩된 값을 api_key 변수에 대입
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


## 데이터 전처리 함수 작성


리뷰 JSON 파일 로드

In [2]:
import json

with open('./res/3.kyongpo_the_hongc.json', 'r') as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다',
  'stars': 5,
  'date': '2025.02.03'},
 {'review': '로비도 고급지고 객실도 깔끔하니 너무나 만족스런1박 했습니다.\n침구가 너무 좋아서 잘 잘수있었어요.\n다음에 여행가면 여기에 다시 묵을겁니다',
  'stars': 5,
  'date': '2024.12.23'},
 {'review': '친절한 안내와 정리정돈도 잘 되어 있어 편안하게 잘수있었어요',
  'stars': 5,
  'date': '2024.12.09'}]

좋은 평점 및 나쁜 평점 정의
- 좋은 평점 = 별 5개 이상
- 나쁜 평점 = 별 4개 이하

In [3]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(329, 91)

좋은 리뷰 및 나쁜 리뷰로 구분 후 저장

In [4]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        # specital token 추가
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        # specital token 추가
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_good[:3]

['[REVIEW_START]강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다[REVIEW_END]',
 '[REVIEW_START]로비도 고급지고 객실도 깔끔하니 너무나 만족스런1박 했습니다.\n침구가 너무 좋아서 잘 잘수있었어요.\n다음에 여행가면 여기에 다시 묵을겁니다[REVIEW_END]',
 '[REVIEW_START]친절한 안내와 정리정돈도 잘 되어 있어 편안하게 잘수있었어요[REVIEW_END]']

하나의 문자열로 합치기

In [5]:
# good 문자, 하나의 문자열로 합치기
reviews_good_text = '\n'.join(reviews_good)
reviews_good_text[:100]

'[REVIEW_START]강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다[REVIEW_END]\n[REVIEW_START]로'

In [6]:
# bad 문자, 하나의 문자열로 합치기
reviews_bad_text = '\n'.join(reviews_bad)
reviews_bad_text[:100]

'[REVIEW_START]방음이 좀 안되긴 한데 욕실 깨끗해서 좋아요[REVIEW_END]\n[REVIEW_START]공사한다고 예약힐때부터 알려주면좋겠고 체크인할때 공사한다고 불편'

리뷰데이터 전처리를 위한 함수 작성

In [7]:
import datetime
from dateutil import parser

def preprocess_reviews(path):
    with open(path, 'r', encoding='utf-8') as f:
        # list 형태로 json 파일 일기
        review_list = json.load(f)

    # good, bad 데이터 저장 변수 초기화
    reviews_good, reviews_bad = [], []

    # 현재 이전 6개월 데이터 사용을 위해 날짜계산
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)
    # print(date_boundary)

    # 리뷰데이터 
    for r in review_list:
        review_date_str = r['date']
        try:
            # 리뷰 날짜 문자열을 datetime 객체로 변환
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            # 정상 날짜 데이터 아닌경우 현재 날짜 대입
            review_date = current_date

        # 날짜 바운더리 보다 작으면, pass
        if review_date < date_boundary:
            continue

        # 평점 5점 기준
        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    # good, bad 하나의 문자열 리스트로 합치기
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    # good 리뷰와 bad 리뷰 반환
    return reviews_good_text, reviews_bad_text

# 리뷰 데이터 전처리 함수 호출
data='./res/3.kyongpo_the_hongc.json'
good, bad = preprocess_reviews(data)
good[:100]

'[REVIEW_START]강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다[REVIEW_END]\n[REVIEW_START]로'

In [8]:
bad[:100]

'[REVIEW_START]방음이 좀 안되긴 한데 욕실 깨끗해서 좋아요[REVIEW_END]\n[REVIEW_START]공사한다고 예약힐때부터 알려주면좋겠고 체크인할때 공사한다고 불편'

### 평가용 함수 작성

평가 기준 설정
- MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
  - 비교하는 방식 vs. 점수 매기는 방식
  - 점수라는게 애매 할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
  - 경험상 점수보다는 비교가 상대적으로 더 정확한 편

평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accommodations)
    - [[A]] – 어시스턴트 A가 더 나을 경우
    - [[B]] – 어시스턴트 B가 더 나을 경우
    - [[C]] – 두 응답이 동등한 품질일 경우

In [10]:
# def pairwise_eval(리뷰원본, AI_a, AI_b):
def pairwise_eval(reviews, answer_a, answer_b):
    # 평가 프롬프트 
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user's instructions and answers the user's question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]

[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]"""
    
    # OpenAI API를 사용하여 평가 요청
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion.choices[0].message.content

### Baseline 응답 개발

In [11]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [12]:
# 높은 평점만 리뷰 데이터만 받음음
reviews, _ = preprocess_reviews(path='./res/3.kyongpo_the_hongc.json')

### 리뷰 데이터 요약 함수 정의 및 실행

In [13]:
def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion.choices[0].message.content

print(summarize(reviews, PROMPT_BASELINE))

강릉 여행 중 예약한 호텔은 주인의 품격이 느껴지며 조용하고 좋았다. 로비와 객실이 고급스럽고 깔끔해서 만족스러운 1박을 했고, 침구가 편안해서 잘 잘 수 있었다. 친절한 안내와 청결한 환경으로 편안하게 머물렀고, 위치도 좋아서 잘 쉬고 갔다. 부모님도 만족하셨고, 직원들도 친절하고 좋았다.


## 응답결과 평가하기

대조군 문장 정의하기

In [14]:
# 대조군 문장 가져오기(야놀자 AI good 요약 문장)
# summary_real_20250530 = '숙소는 강릉 경포해변 바로 앞에 위치하며, 오션뷰와 경포호수 전망을 제공합니다. 객실과 화장실이 청결하게 관리되고 있으며, 편안한 침대와 고급 침구로 쾌적한 숙박 환경을 제공합니다. 주변에 식당과 편의시설이 가까워 접근성이 좋고, 주차 공간도 넓습니다. 직원들의 친절한 응대와 서비스도 긍정적인 평가를 받고 있으며, 대체로 소음 차단이 잘 이루어져 조용한 환경에서 휴식을 취할 수 있습니다. 전반적으로 방문자들은 숙소의 청결도와 편안함, 아늑한 분위기에 매우 만족하는 후기를 남기고 있습니다.'

summary_4o_20250530 = summarize(reviews, PROMPT_BASELINE, temperature=0.0, model='gpt-4o-2024-05-13')
summary_4o_20250530

'이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용해서 편안하게 쉴 수 있는 곳으로 추천됩니다. 로비와 객실이 고급스럽고 깔끔하다는 점도 언급되었습니다. 다만, 배달음식 이용이 제한된다는 점이 아쉽다는 의견이 있습니다. 전반적으로 재방문 의사가 많고 만족도가 높은 숙소입니다.'

In [15]:
# 비교하기
# pairwise_eval(리뷰 데이터, AI 요약한 데이터_A, 대조군 AI 문장_B))
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE), summary_4o_20250530))

Both Assistant A and Assistant B provided summaries that capture the essence of the user reviews. However, there are some differences in their approaches and the details they included.

1. **Helpfulness and Relevance**:
   - **Assistant A**: Provides a more narrative summary, capturing the overall experience of the guests, including the quality of the hotel, the cleanliness, the friendliness of the staff, and the good location.
   - **Assistant B**: Offers a more structured summary, highlighting specific positive aspects such as cleanliness, good bedding, friendly staff, and good location. It also mentions a specific negative point about the restriction on delivery food.

2. **Accuracy**:
   - Both assistants accurately reflect the positive feedback about cleanliness, friendly staff, good location, and overall satisfaction. However, Assistant B includes a specific negative point about the restriction on delivery food, which is mentioned in one of the reviews.

3. **Depth and Detail**:


## 대규모 평가 스크립트
- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [25]:
# 같은 리뷰에 대해서 요약을 10번 수행(일반적으로는 50~100번 수행하면 좋음)
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0) for _ in range(eval_count)]
summaries_baseline

['강릉 여행 중 예약한 호텔은 주인의 품격이 느껴지며 조용하고 좋았다. 로비와 객실이 고급스럽고 깔끔하여 만족스러웠고, 친절한 안내와 정리가 잘 되어 있어 편안하게 숙박했다. 숙소는 깨끗하고 위치가 좋아 쉬기에 좋았으며, 부모님도 만족했다고 한다. 다만 외부 음식 관련 사전 안내가 필요하다고 하는 글도 있으니 주의가 필요하다.',
 '강릉 여행 중 예약한 숙소는 주인의 품격을 느낄 수 있는 조용하고 깔끔한 곳이었고, 친절한 안내와 청결한 시설로 만족스러웠다. 위치도 좋고 침구도 편안하여 잘 쉴 수 있었으며, 직원들의 친절함과 완벽한 서비스로 부모님도 만족했다. 잘정돈된 객실과 깨끗한 시설로 편안한 숙박을 경험할 수 있었고, 조식 샌드위치도 맛있어 재방문 의사가 생겼다. 위치가 편리하고 깔끔한 숙소로 푹 쉬고 나다가, 포근하고 깨끗한 숙소에서 편안한 휴식을 취할 수 있었고, 직원들의 친절함과 청결한 시설에 감동했다.',
 '강릉 여행길에 예약을 잘한 호텔로, 주인의 품격이 느껴지며 조용하고 좋은 곳. 대부분의 리뷰에서 친절한 서비스와 깨끗한 시설에 만족을 표현하는 내용이 반복되며, 몇몇 리뷰에서는 외부 음식 주문이 어려워 아쉬움을 표현하는 것도 있음. 전체적으로 편안하고 만족스러운 투숙 경험을 강조하는 리뷰들이 많음.',
 '강릉 여행 중 예약한 호텔은 주인의 품격이 느껴지며 조용하고 좋았다. 로비와 객실이 고급스럽고 깔끔해서 만족스러웠고, 침구도 편안해서 잘 잤다. 부모님도 만족하셨고, 직원들의 친절에 감사하며 깨끗하고 혼자 사용하기에 넓어서 좋았다. 위치가 좋고 주변 인프라가 편리해서 추천한다.',
 '강릉 여행동안 예약 잘한 조용하고 고급스러운 호텔, 깔끔한 객실과 침구로 만족스러운 1박, 친절한 안내와 청결한 숙박, 위치 좋고 깨끗한 곳으로 잘 쉬었음, 가족 여행에도 만족스러운 숙소 묵음.',
 '강릉 여행길에 예약을 잘한 호텔로 품격이 느껴졌고 조용하고 깨끗한 객실에 만족했습니다. 다음에도 또 묵고 싶은 친절하고 청결한 숙소예요.',
 '강릉 여행 중

In [26]:
pairwise_words = [summary_4o_20250530 for _ in range(len(summaries_baseline))]
pairwise_words

['이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용해서 편안하게 쉴 수 있는 곳으로 추천됩니다. 로비와 객실이 고급스럽고 깔끔하다는 점도 언급되었습니다. 다만, 배달음식 이용이 제한된다는 점이 아쉽다는 의견이 있습니다. 전반적으로 재방문 의사가 많고 만족도가 높은 숙소입니다.',
 '이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용해서 편안하게 쉴 수 있는 곳으로 추천됩니다. 로비와 객실이 고급스럽고 깔끔하다는 점도 언급되었습니다. 다만, 배달음식 이용이 제한된다는 점이 아쉽다는 의견이 있습니다. 전반적으로 재방문 의사가 많고 만족도가 높은 숙소입니다.',
 '이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용해서 편안하게 쉴 수 있는 곳으로 추천됩니다. 로비와 객실이 고급스럽고 깔끔하다는 점도 언급되었습니다. 다만, 배달음식 이용이 제한된다는 점이 아쉽다는 의견이 있습니다. 전반적으로 재방문 의사가 많고 만족도가 높은 숙소입니다.',
 '이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용해서 편안하게 쉴 수 있는 곳으로 추천됩니다. 로비와 객실이 고급스럽고 깔끔하다는 점도 언급되었습니다. 다만, 배달음식 이용이 제한된다는 점이 아쉽다는 의견이 있습니다. 전반적으로 재방문 의사가 많고 만족도가 높은 숙소입니다.',
 '이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용해서 편안하게 쉴 수 있는 곳으로 추천됩니다. 로비와 객실이 고급스럽고 깔끔하다는 점도 언급되었습니다. 다만, 배달음식 이용이 제한된다는 점이 아쉽다는 의견이 있습니다. 전반적으로 재방문 의사가 많고 만족도가 높은 숙소입니다.',
 '이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용해서 편안하게 쉴 수 있는 곳으로 추천됩니다.

- 대규모 데이터를 비교평가

In [27]:

from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        # 최종 결과
        verdict_text = pairwise_eval(reviews, answers_a[i], answers_b[i])

        # A인경우 카운트
        if '[[A]]' in verdict_text:
            a_cnt += 1
        # B인경우 카운트
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        # C인경우 카운트
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        # 나머지
        else:
            print('Evaluation Error')
    # a good, b bad, c 무승부 비슷함.
    return a_cnt, b_cnt, draw_cnt

# 요약한 것과 대조군 비교
wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_4o_20250530 for _ in range(len(summaries_baseline))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:44<00:00,  4.43s/it]

Wins: 4, Losses: 6, Ties: 0


### 응답결과 고도화 1 - 요약 조건들 명시 후 평가
- role과 수행 목표 정의
- 요약 결과에 대한 충족 조건
    - 톤엔 매너와 예시
    - 요약 결과에 갯수

In [28]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0) for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:51<00:00,  5.10s/it]

Wins: 9, Losses: 1, Ties: 0


### 응답결과 고도화 2 - 입력 데이터의 품질 증가
- 리뷰 길이가 짧은 것 필터링함, 고품질 리뷰만 남김

In [30]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/3.kyongpo_the_hongc.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    filtered_cnt = 0
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue

        # 고품질 리뷰만 남김
        if len(r['review']) < 30:
            filtered_cnt += 1
            # print(r['review'])
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    # 최대 길이가 50까지 되도로 자름 
    reviews_good = reviews_good[:min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()

In [31]:
# 평가 테스트
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125') for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [01:38<00:00,  9.84s/it]

Wins: 9, Losses: 1, Ties: 0


### 응답결과 고도화 3 - One-Shot Prompting

In [32]:
reviews_1shot, _ = preprocess_reviews('./res/3.kyongpo_the_hongc.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-3.5-turbo-0125')
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125') for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:38<00:00,  3.82s/it]

Wins: 9, Losses: 1, Ties: 0


In [33]:
summaries

['강릉 여행길에 들르면 아주 예약을 잘한 호텔이었습니다. 주인의 품격이 느껴지고 조용하고 좋았어요. 로비도 고급지고 객실도 깔끔해서 너무 만족스러운 1박이었고 다음에 여행가면 다시 묵을 것입니다. 친절한 안내와 정리정돈도 잘 되어 있어 편안하게 잘 수 있었습니다. 부모님께 예약해드린 숙소에서 너무 만족하셨다고 합니다. 객실이 깨끗하고 편안해서 좋았고 직원분들도 너무 친절하셨습니다.',
 '강릉 여행길에 드러놓은 숙소는 예약을 잘한 것 같습니다. 주인의 품격이 느껴지며 조용하고 좋은 분위기를 자랑합니다. 홀과 객실이 고급스러워 만족스러운 1박을 보낸 여행객도 있었고, 친절한 안내와 청결한 환경으로 편안한 숙박을 경험한 다른 손님들도 계셨습니다.전체적으로 만족스러운 숙소로 다양한 리뷰들이 모두 좋은 평가를 받았습니다.',
 '강릉 여행길에 들르게 되어 아주 예약을 잘한 것 같아요. 호텔에서는 주인의 품격이 느껴졌고, 조용하고 편안했어요. 로비도 고급스럽고 객실도 깔끔해서 너무 만족스러운 1박이었어요. 침구가 너무 좋아서 잠도 편안했고, 다음에 또 방문할 의향이 있습니다. 요구사항에 따라 외부 음식 관련 안내가 필요한 것이 아쉬운 점이지만, 시설은 깨끗하고 편안하고 주변 인프라가 좋아 추천할 만한 숙소입니다. 직원분들도 친절하고 청결하며 기분 좋은 시간을 보낼 수 있어 좋았어요.',
 '강릉 여행길에 들렸는데 아주 예약을 잘한 것 같아요. 호텔에서는 주인의 품격이 느껴져서 조용하고 좋았어요. 다음에 여행을 또 온다면 여기에 다시 머무르고 싶을 정도로 만족스러운 숙소였습니다. 부모님도 너무 만족하셨다고 하셔서 기뻐요. 쾌적하고 아늑하며 침구류도 깨끗했어요. 프론트 데스크 분들도 친절하시고 체크인 시간보다 일찍 도착해도 빠르게 처리해주셔서 감사했어요. 종합적으로 깨끗하고 편안한 시설, 친절한 서비스로 추천할 만한 숙소였습니다.',
 '강릉 여행길에 들렀는데 아주 예약을 잘한 숙소입니다. 호텔에서는 주인의 품격이 느껴지며, 조용하고 좋은 분위기를 느끼실 수 있습니다. 

### 응답결과 고도화 3 - one-Shot Prompting

In [34]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo-0125') for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:39<00:00,  3.99s/it]

Wins: 10, Losses: 0, Ties: 0


### prompt_1shot을 피클파일로 저장하기

In [35]:
# prompt_1shot을 피클파일로 저장하기
file_name = "./res/prompt_1shotv1.pickle"
# 파일로 저장
import pickle
with open(file_name, 'wb') as f:
    pickle.dump(prompt_1shot, f)

In [36]:
print(summaries)

['강릉 여행길에 예약을 잘한 호텔로 주인의 품격과 조용한 환경이 좋았습니다. 로비와 객실이 고급스럽고 깔끔해서 만족스러운 1박이었으며, 침구도 훌륭하여 편안한 휴식을 취할 수 있었습니다. 친절한 안내와 정리정돈이 편안한 숙박을 도와주었고, 부모님도 만족해하셨습니다. 또한 직원들의 친절함과 깨끗한 객실로 편안한 경험을 하였습니다.', '강릉 여행길에 들렀을 때 아주 예약을 잘한 호텔이었습니다. 주인의 품격이 느껴지며 조용하고 좋은 숙소였습니다. 로비도 고급스럽고 객실도 깔끔하여 너무 만족스러운 1박을 했습니다. 침구가 훌륭해서 편안하게 잠을 잘 수 있었고, 다음에 여행을 가면 다시 묵을 의향이 있습니다. 친절한 안내와 정리정돈이 잘 되어 있어 편안하게 머물렀습니다. 부모님도 만족하셨고, 깨끗하고 아늑한 숙소에서 편안한 시간을 보낼 수 있었습니다. 객실은 깨끗하고 편안하며, 직원분들도 친절하여 좋은 경험을 하였습니다. 숙소 위치가 교동에 있어 이동이 편리하며, 주차는 주차타워를 사용해야 하는 것이 조금 단점이지만 출차는 편리하고 전체적으로 깔끔하고 좋은 숙소라고 추천합니다. 따뜻하고 깨끗한 숙소에서 좋은 경험을 했으며, 다음에도 또 방문할 의향이 있습니다.', '강릉 여행길에 들렀을 때 아주 예약을 잘한 호텔이었습니다. 주인의 품격이 느껴지며 조용하고 좋은 숙소였습니다. 로비도 고급스럽고 객실도 깔끔하여 너무 만족스러운 1박을 했습니다. 침구가 훌륭해서 편안하게 잠을 잘 수 있었고, 다음에 여행을 가면 다시 묵을 의향이 있습니다. 친절한 안내와 정리정돈이 잘 되어 있어 편안하게 머물렀습니다. 부모님도 만족하셨고, 깨끗하고 아늑한 숙소에서 편안한 시간을 보낼 수 있었습니다. 객실은 깨끗하고 편안하며, 직원분들도 친절하여 좋은 경험을 하였습니다.', '강릉 여행길에 들렀을 때 아주 예약을 잘한 호텔이었습니다. 주인의 품격이 느껴지며 조용하고 좋은 숙소였습니다. 로비도 고급스럽고 객실도 깔끔하여 너무 만족스러운 1박을 했습니다. 침구가 훌륭해서 편안하게 잠을 잘 수 